In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://mongodb:27017/")

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("myApp") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.2') \
.getOrCreate()

In [4]:
sars = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri",
"mongodb://mongodb:27017/epidemics.sars").load()

In [5]:
sars.createOrReplaceTempView("sars")

In [33]:
sars = spark.sql("SELECT _id, country, date_format(date, 'YYYY-MM-dd') as date, \
COALESCE(int((deaths - lag(deaths,1) over (partition by country ORDER BY date asc))), deaths) as deaths, deaths as cumul_deaths, \
COALESCE(int((cumul_nb_cases - lag(cumul_nb_cases,1) over (partition by country ORDER BY date asc))), cumul_nb_cases) as cases, \
cumul_nb_cases as cumul_cases FROM sars")

In [34]:
sars.show()

+--------------------+-------+----------+------+------------+-----+-----------+
|                 _id|country|      date|deaths|cumul_deaths|cases|cumul_cases|
+--------------------+-------+----------+------+------------+-----+-----------+
|[5eaf225b9754f390...| Sweden|2003-04-14|     0|           0|    1|          1|
|[5eaf225b9754f390...| Sweden|2003-04-15|     0|           0|    0|          1|
|[5eaf225b9754f390...| Sweden|2003-04-16|     0|           0|    0|          1|
|[5eaf225b9754f390...| Sweden|2003-04-17|     0|           0|    0|          1|
|[5eaf225b9754f390...| Sweden|2003-04-18|     0|           0|    2|          3|
|[5eaf225b9754f390...| Sweden|2003-04-19|     0|           0|    0|          3|
|[5eaf225b9754f390...| Sweden|2003-04-21|     0|           0|    0|          3|
|[5eaf225b9754f390...| Sweden|2003-04-22|     0|           0|    0|          3|
|[5eaf225b9754f390...| Sweden|2003-04-23|     0|           0|    0|          3|
|[5eaf225b9754f390...| Sweden|2003-04-24

In [18]:
#sars = spark.sql("SELECT _id, country, date_format(date, 'YYYY-MM-dd') as date, deaths, cumul_nb_cases as cumul_cases FROM sars")
#sars.show()

In [35]:
sars.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://mongodb:27017/epidemics2.epidemics_sars").save()